## 1 样本与标注  

### Display Samples  

Mask R-CNN 中使用的训练数据是 COCO 数据集，Mask 的标注了物体所在的位置及边缘信息，不像物体检测任务中，物体位置标注的是一个矩形框  

### Bounding Boxes  

没有使用源数据集中用于目标检测的边界框标注坐标，而是使用 Mask 来计算边界框。从而可以在不同的源数据集上，统一的处理边界框，而且，因为只是从更新后的 Mask 生成边界框，而不用根据不同的转换去调整边界框的坐标，所以还可以更轻松地调整大小，旋转或裁剪图像。  


### Image Resize  

为了支持每批次的多个图像，图像被调整为一个统一的尺寸（1024x1024）。保持图像的长宽比，如果图像不是正方形，则在顶部/底部或者左/右进行零填充。  


### Mini Masks  

使用高分辨率图像进行训练时，实例二进制 Mask 可能会变大。例如，如果使用 1024x1024 图像训练，那么单个实例的 Mask 需要 1MB 的内存。如果一个图像有 100 个实例，那么就有 100MB 的Mask。  

为了提高训练速度，我们通过以下方式优化 Mask：  
* 我们存储目标边界框内的 Mask 像素，而不是完整图像的 Mask 。大多数物体与图像大小相比明显较小，所以我们通过不在物体周围存储很多零来节省空间。  
* 我们将 Mask 调整为较小的尺寸（例如56x56）。对于大于所选大小的对象，会失去一些准确性。但是大多数对象标注并不是非常准确，所以对于大多数情况来说，这种损失是可以忽略的。  

为了可视化掩码大小调整的效果，并验证代码的正确性，可以看到一些示例，如下所示。  


### Anchors

anchors 的顺序很重要。在训练和预测阶段使用相同的顺序。它必须匹配卷积执行的顺序。  

对于一个 FPN 网络来说，anchors 的排列方式必须能够很容易地将 anchors 与预测 anchors 分数和移位的卷积层输出相匹配。  
* 先按金字塔等级排序。 第一级的所有 anchors，然后是第二级的所有 anchors，等等。这使得通过级别分离 anchors 更容易。  
* 每层中，按照特征图处理的顺序对 anchors 进行排序。通常，卷积层从左上角开始并向右逐行移动的处理特征图。
* 对于每个特征图单元格，为不同比例的 anchors 选择排序顺序。这里我们以传递给函数的比例顺序来对 anchors 进行排序。  


**Anchor Stride:**

在 FPN 架构中，前几层的特征映射是高分辨率的。例如，如果输入图像是1024x1024，那么第一层的特征图是 256x256，产生大约 200K 的 anchors（256 * 256 * 3）。这些 anchors 是 32×32 像素，它们相对于图像像素的步幅是 4 个像素，因此有很多重叠。如果我们以一定的步长为特征地图中的单元格生成 anchors ，可以显著的减少负载。例如，2 的步幅将削减四倍的 anchors 数量。  


**Show positive anchors**  

**Show negative anchors**  

**Show neutral anchors**  

## 2 处理流程  

### Stage 1: Region Proposal Network  

The Region Proposal Network (RPN) runs a lightweight binary classifier on a lot of boxes (anchors) over the image and returns object/no-object scores. Anchors with high objectness score (positive anchors) are passed to the stage two to be classified.

Often, even positive anchors don't cover objects fully. So the RPN also regresses a refinement (a delta in location and size) to be applied to the anchors to shift it and resize it a bit to the correct boundaries of the object.  

#### 1.a RPN Targets  

The RPN targets are the training values for the RPN. To generate the targets, we start with a grid of anchors that cover the full image at different scales, and then we compute the IoU of the anchors with ground truth object. Positive anchors are those that have an IoU >= 0.7 with any ground truth object, and negative anchors are those that don't cover any object by more than 0.3 IoU. Anchors in between (i.e. cover an object by IoU >= 0.3 but < 0.7) are considered neutral and excluded from training.

To train the RPN regressor, we also compute the shift and resizing needed to make the anchor cover the ground truth object completely.  

#### 1.b RPN Predictions  

Here we run the RPN graph and display its predictions.  

### Stage 2: Proposal Classification  

This stage takes the region proposals from the RPN and classifies them.  

#### 2.a Proposal Classification  

Run the classifier heads on proposals to generate class propbabilities and bounding box regressions.  

#### 2.c Step by Step Detection  

Here we dive deeper into the process of processing the detections.  

Apply Bounding Box Refinement  

Filter Low Confidence Detections  

Per-Class Non-Max Suppression  

### Stage 3: Generating Masks  

This stage takes the detections (refined bounding boxes and class IDs) from the previous layer and runs the mask head to generate segmentation masks for every instance.  

#### 3.a Mask Targets

These are the training targets for the mask branch  

#### 3.b Predicted Masks  

